In [1]:
# !pip install pulp

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ------ --------------------------------- 2.6/16.4 MB 13.7 MB/s eta 0:00:02
   ------------- -------------------------- 5.5/16.4 MB 12.9 MB/s eta 0:00:01
   ------------------ --------------------- 7.6/16.4 MB 12.7 MB/s eta 0:00:01
   ----------------------- ---------------- 9.7/16.4 MB 11.6 MB/s eta 0:00:01
   --------------------------- ------------ 11.3/16.4 MB 10.8 MB/s eta 0:00:01
   --------------------------------- ------ 13.9/16.4 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.4 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 10.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# data_path='/content/drive/MyDrive/drive-download-20230803T045612Z-001/data.csv'
data=pd.read_csv("data.csv", encoding='cp949')
data.head()

,수요지 id,X,Y,초등학령인구
0,C1,126.371505,37.463782,1
1,C2,126.479505,37.495782,46
2,C3,126.481505,37.499782,36
3,C4,126.482505,37.495782,81
4,C5,126.483505,37.497782,25


In [8]:
# target_path='/content/drive/MyDrive/drive-download-20230803T045612Z-001/target.csv'
target=pd.read_csv("target.csv", encoding='cp949')
target.head()

,학교 id,X,Y
0,F1,126.543921,37.467292
1,F2,126.568726,37.496302
2,F3,126.492770,37.484428
3,F4,126.629683,37.495781
4,F5,126.528286,37.450285


In [10]:
# dangerous_middle_path='/content/drive/MyDrive/location_middle.csv'
dm=pd.read_csv("location_middle.csv", encoding='cp949')
# dangerous_east_path='/content/drive/MyDrive/location_east.csv'
de=pd.read_csv("location_east.csv", encoding='cp949')

In [11]:
entertainment_df = pd.concat([dm, de], ignore_index=True)
entertainment_df

,열1,업소명,소재지,위도,경도,Column1
0,0.0,위스키메리(외국인전용),인천광역시 중구 개항로 5,37.472756,126.634241,NaN
1,1.0,디가디가노래타운,인천광역시 중구 홍예문로98번길 9,37.476762,126.630116,NaN
2,2.0,향도 노래천국,인천광역시 중구 서해대로 462-3,37.429981,126.614170,NaN
3,3.0,거래처,인천광역시 중구 신포로27번길 32,37.472794,126.622593,NaN
4,4.0,미,인천광역시 중구 신포로23번길 32,37.472505,126.622338,NaN
...,...,...,...,...,...,...
145,NaN,멍텅구리 단란주점,인천광역시 동구 샛골로 170,37.478703,126.645174,4.0
146,NaN,스카렛,인천광역시 동구 화도진로 68,37.475827,0.000000,5.0
147,NaN,지중해,인천광역시 동구 샛골로 158-2,37.478703,126.645174,6.0
148,NaN,쇼노래단란주점,"인천광역시 동구 샛골로 143,",37.478703,126.645174,7.0


In [12]:
# 각 시설물 입지점에서 유흥시설까지의 거리를 계산하는 함수
def is_within_200m_of_entertainment(target_lat, target_lon, entertainment_df):
    for _, row in entertainment_df.iterrows():
        distance = haversine_distance(target_lat, target_lon, row['위도'], row['경도'])
        if distance <= 0.2:  # 200m로 변환
            return True
    return False

In [13]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (킬로미터)

    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # 위도와 경도의 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # 허버사인 공식
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c

    return distance

In [14]:
s=data['초등학령인구'].sum()
s/750

13.614666666666666

In [15]:
s/200

51.055

##3km 이내

In [16]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 할당

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 3km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (3 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

Status: Optimal
수요지 2는 시설물 입지점 45에 할당됨
수요지 3는 시설물 입지점 45에 할당됨
수요지 4는 시설물 입지점 45에 할당됨
수요지 5는 시설물 입지점 45에 할당됨
수요지 6는 시설물 입지점 45에 할당됨
수요지 7는 시설물 입지점 45에 할당됨
수요지 8는 시설물 입지점 45에 할당됨
수요지 9는 시설물 입지점 45에 할당됨
수요지 10는 시설물 입지점 45에 할당됨
수요지 11는 시설물 입지점 45에 할당됨
수요지 12는 시설물 입지점 45에 할당됨
수요지 13는 시설물 입지점 3에 할당됨
수요지 14는 시설물 입지점 45에 할당됨
수요지 15는 시설물 입지점 45에 할당됨
수요지 16는 시설물 입지점 45에 할당됨
수요지 17는 시설물 입지점 45에 할당됨
수요지 18는 시설물 입지점 45에 할당됨
수요지 19는 시설물 입지점 45에 할당됨
수요지 20는 시설물 입지점 45에 할당됨
수요지 21는 시설물 입지점 45에 할당됨
수요지 22는 시설물 입지점 45에 할당됨
수요지 23는 시설물 입지점 3에 할당됨
수요지 24는 시설물 입지점 45에 할당됨
수요지 25는 시설물 입지점 44에 할당됨
수요지 26는 시설물 입지점 3에 할당됨
수요지 27는 시설물 입지점 44에 할당됨
수요지 28는 시설물 입지점 3에 할당됨
수요지 29는 시설물 입지점 44에 할당됨
수요지 30는 시설물 입지점 44에 할당됨
수요지 31는 시설물 입지점 45에 할당됨
수요지 32는 시설물 입지점 45에 할당됨
수요지 33는 시설물 입지점 40에 할당됨
수요지 34는 시설물 입지점 45에 할당됨
수요지 35는 시설물 입지점 40에 할당됨
수요지 36는 시설물 입지점 44에 할당됨
수요지 37는 시설물 입지점 45에 할당됨
수요지 38는 시설물 입지점 45에 할당됨
수요지 39는 시설물 입지점 40에 할당됨
수요지 40는 시설물 입지점 37에 할당됨
수요지 41는 시설물 입지점 37에 할당됨
수요지 42는 시설물 입지점 37에 할당됨
수요지 43는 시설물 

##2.8km 이내

In [17]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 할당

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 2.8km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (2.8 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

Status: Optimal
수요지 2는 시설물 입지점 45에 할당됨
수요지 3는 시설물 입지점 45에 할당됨
수요지 4는 시설물 입지점 45에 할당됨
수요지 5는 시설물 입지점 45에 할당됨
수요지 6는 시설물 입지점 45에 할당됨
수요지 7는 시설물 입지점 45에 할당됨
수요지 8는 시설물 입지점 45에 할당됨
수요지 9는 시설물 입지점 45에 할당됨
수요지 10는 시설물 입지점 45에 할당됨
수요지 11는 시설물 입지점 45에 할당됨
수요지 12는 시설물 입지점 45에 할당됨
수요지 13는 시설물 입지점 45에 할당됨
수요지 14는 시설물 입지점 45에 할당됨
수요지 15는 시설물 입지점 45에 할당됨
수요지 16는 시설물 입지점 45에 할당됨
수요지 17는 시설물 입지점 45에 할당됨
수요지 18는 시설물 입지점 45에 할당됨
수요지 19는 시설물 입지점 45에 할당됨
수요지 20는 시설물 입지점 45에 할당됨
수요지 21는 시설물 입지점 45에 할당됨
수요지 22는 시설물 입지점 45에 할당됨
수요지 23는 시설물 입지점 46에 할당됨
수요지 24는 시설물 입지점 45에 할당됨
수요지 25는 시설물 입지점 46에 할당됨
수요지 26는 시설물 입지점 44에 할당됨
수요지 27는 시설물 입지점 44에 할당됨
수요지 28는 시설물 입지점 44에 할당됨
수요지 29는 시설물 입지점 46에 할당됨
수요지 30는 시설물 입지점 44에 할당됨
수요지 31는 시설물 입지점 46에 할당됨
수요지 32는 시설물 입지점 46에 할당됨
수요지 33는 시설물 입지점 40에 할당됨
수요지 34는 시설물 입지점 46에 할당됨
수요지 35는 시설물 입지점 40에 할당됨
수요지 36는 시설물 입지점 44에 할당됨
수요지 37는 시설물 입지점 46에 할당됨
수요지 38는 시설물 입지점 46에 할당됨
수요지 39는 시설물 입지점 40에 할당됨
수요지 40는 시설물 입지점 46에 할당됨
수요지 41는 시설물 입지점 46에 할당됨
수요지 42는 시설물 입지점 46에 할당됨
수요지 43는 

##2.6km이내

In [18]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 할당

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 2.6km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (2.6 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

Status: Optimal
수요지 2는 시설물 입지점 45에 할당됨
수요지 3는 시설물 입지점 45에 할당됨
수요지 4는 시설물 입지점 45에 할당됨
수요지 5는 시설물 입지점 45에 할당됨
수요지 6는 시설물 입지점 45에 할당됨
수요지 7는 시설물 입지점 45에 할당됨
수요지 8는 시설물 입지점 45에 할당됨
수요지 9는 시설물 입지점 45에 할당됨
수요지 10는 시설물 입지점 45에 할당됨
수요지 11는 시설물 입지점 45에 할당됨
수요지 12는 시설물 입지점 45에 할당됨
수요지 13는 시설물 입지점 45에 할당됨
수요지 14는 시설물 입지점 45에 할당됨
수요지 15는 시설물 입지점 45에 할당됨
수요지 16는 시설물 입지점 45에 할당됨
수요지 17는 시설물 입지점 45에 할당됨
수요지 18는 시설물 입지점 45에 할당됨
수요지 19는 시설물 입지점 45에 할당됨
수요지 20는 시설물 입지점 45에 할당됨
수요지 21는 시설물 입지점 45에 할당됨
수요지 22는 시설물 입지점 45에 할당됨
수요지 23는 시설물 입지점 46에 할당됨
수요지 24는 시설물 입지점 45에 할당됨
수요지 25는 시설물 입지점 46에 할당됨
수요지 26는 시설물 입지점 44에 할당됨
수요지 27는 시설물 입지점 44에 할당됨
수요지 28는 시설물 입지점 44에 할당됨
수요지 29는 시설물 입지점 46에 할당됨
수요지 30는 시설물 입지점 44에 할당됨
수요지 31는 시설물 입지점 46에 할당됨
수요지 32는 시설물 입지점 46에 할당됨
수요지 33는 시설물 입지점 40에 할당됨
수요지 34는 시설물 입지점 46에 할당됨
수요지 35는 시설물 입지점 40에 할당됨
수요지 36는 시설물 입지점 44에 할당됨
수요지 37는 시설물 입지점 46에 할당됨
수요지 38는 시설물 입지점 46에 할당됨
수요지 39는 시설물 입지점 40에 할당됨
수요지 40는 시설물 입지점 46에 할당됨
수요지 41는 시설물 입지점 46에 할당됨
수요지 42는 시설물 입지점 46에 할당됨
수요지 43는 

##2.4km 이내

In [19]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])


# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 2.4km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (2.4 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

Status: Optimal
수요지 2는 시설물 입지점 45에 할당됨
수요지 3는 시설물 입지점 45에 할당됨
수요지 4는 시설물 입지점 45에 할당됨
수요지 5는 시설물 입지점 45에 할당됨
수요지 6는 시설물 입지점 45에 할당됨
수요지 7는 시설물 입지점 45에 할당됨
수요지 8는 시설물 입지점 45에 할당됨
수요지 9는 시설물 입지점 45에 할당됨
수요지 10는 시설물 입지점 45에 할당됨
수요지 11는 시설물 입지점 45에 할당됨
수요지 12는 시설물 입지점 45에 할당됨
수요지 13는 시설물 입지점 45에 할당됨
수요지 14는 시설물 입지점 45에 할당됨
수요지 15는 시설물 입지점 45에 할당됨
수요지 16는 시설물 입지점 45에 할당됨
수요지 17는 시설물 입지점 45에 할당됨
수요지 18는 시설물 입지점 45에 할당됨
수요지 19는 시설물 입지점 45에 할당됨
수요지 20는 시설물 입지점 45에 할당됨
수요지 21는 시설물 입지점 45에 할당됨
수요지 22는 시설물 입지점 45에 할당됨
수요지 23는 시설물 입지점 46에 할당됨
수요지 24는 시설물 입지점 45에 할당됨
수요지 25는 시설물 입지점 46에 할당됨
수요지 26는 시설물 입지점 44에 할당됨
수요지 27는 시설물 입지점 44에 할당됨
수요지 28는 시설물 입지점 44에 할당됨
수요지 29는 시설물 입지점 46에 할당됨
수요지 30는 시설물 입지점 44에 할당됨
수요지 31는 시설물 입지점 46에 할당됨
수요지 32는 시설물 입지점 46에 할당됨
수요지 33는 시설물 입지점 40에 할당됨
수요지 34는 시설물 입지점 46에 할당됨
수요지 35는 시설물 입지점 40에 할당됨
수요지 36는 시설물 입지점 44에 할당됨
수요지 37는 시설물 입지점 46에 할당됨
수요지 38는 시설물 입지점 46에 할당됨
수요지 39는 시설물 입지점 40에 할당됨
수요지 40는 시설물 입지점 46에 할당됨
수요지 41는 시설물 입지점 46에 할당됨
수요지 42는 시설물 입지점 46에 할당됨
수요지 43는 

In [20]:
# 수요지1과 설치된 시설물 입지점 사이의 거리를 저장할 리스트
distances_from_demand_1 = []

# 수요지1과 각 설치된 시설물 입지점 사이의 거리 계산
for j in range(num_j):
    if k[j].varValue == 1:  # 설치된 시설물 입지점만 고려
        distances_from_demand_1.append((j+1, b[0][j]))  # (시설물 입지점 인덱스, 거리)

# 가장 가까운 거리를 갖는 시설물 입지점 찾기
nearest_facility_index, nearest_distance = min(distances_from_demand_1, key=lambda x: x[1])

print(f"수요지1과 가장 가까운 설치된 시설물 입지점은 {nearest_facility_index}이며, 거리는 {nearest_distance:.2f}km입니다.")


수요지1과 가장 가까운 설치된 시설물 입지점은 45이며, 거리는 10.90km입니다.


In [21]:
# 수요지 1에 대한 정보 업데이트

# 1. assigned_df 업데이트
# 수요지 1과 가장 가까운 설치된 시설물 입지점을 연결
facility_row_in_assigned_df = assigned_df[assigned_df['시설물_입지점'] == nearest_facility_index]
if not facility_row_in_assigned_df.empty:
    # 해당 시설물 입지점에 이미 연결된 수요지가 있으면 리스트에 수요지 1 추가
    facility_row_index = facility_row_in_assigned_df.index[0]
    assigned_df.at[facility_row_index, '연결된_수요지'].append(1)
else:
    # 해당 시설물 입지점에 연결된 수요지가 없으면 새로운 행 추가
    assigned_df = assigned_df.append({'시설물_입지점': nearest_facility_index, '연결된_수요지': [1]}, ignore_index=True)

# 2. facility_demand_df 업데이트
# 수요지 1의 수요량을 찾기
demand_1_value = a[0]

# 수요지 1의 수요량을 가장 가까운 설치된 시설물 입지점의 총 수요량에 추가
facility_row_in_facility_demand_df = facility_demand_df[facility_demand_df['시설물_입지점'] == nearest_facility_index]
if not facility_row_in_facility_demand_df.empty:
    # 해당 시설물 입지점에 이미 수요량이 기록되어 있으면 수요량 추가
    facility_row_index = facility_row_in_facility_demand_df.index[0]
    facility_demand_df.at[facility_row_index, '배정된_수요량'] += demand_1_value

# 업데이트된 데이터프레임 출력
print(assigned_df)
print(facility_demand_df)


    시설물_입지점                                            연결된_수요지
0         2                               [87, 90, 91, 92, 93]
1         8  [250, 251, 267, 274, 275, 279, 280, 281, 286, ...
2        12  [94, 95, 96, 97, 98, 99, 103, 104, 105, 106, 1...
3        21  [178, 179, 191, 192, 201, 202, 203, 204, 209, ...
4        24  [133, 134, 143, 150, 151, 152, 158, 159, 160, ...
5        28                     [100, 101, 102, 116, 138, 219]
6        31  [288, 292, 293, 294, 295, 297, 298, 299, 301, ...
7        33                           [74, 80, 81, 83, 86, 88]
8        35                                   [79, 82, 84, 89]
9        37  [45, 46, 47, 49, 50, 51, 52, 53, 55, 56, 57, 5...
10       40                                       [33, 35, 39]
11       44                               [26, 27, 28, 30, 36]
12       45  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...
13       46  [23, 25, 29, 31, 32, 34, 37, 38, 40, 41, 42, 4...
14       48                       [48, 54, 68, 75, 77, 

In [22]:
# 데이터프레임을 csv로 저장
assigned_df.to_csv('assigned_data1.csv', index=False)
facility_demand_df.to_csv('facility_demand_data1.csv', index=False)

In [24]:
submission_path='submission.csv'
submission=pd.read_csv(submission_path, encoding = 'cp949')
submission

,수요지 id,학교 id,할당 학령인구
0,C1,F1,0
1,C1,F2,0
2,C1,F3,0
3,C1,F4,0
4,C1,F5,0
...,...,...,...
15545,C311,F46,0
15546,C311,F47,0
15547,C311,F48,0
15548,C311,F49,0


In [25]:
# submission 데이터 프레임 업데이트
for _, row in assigned_df.iterrows():
    facility_id = row['시설물_입지점']  # 학교 id
    connected_demands = row['연결된_수요지']  # 해당 학교에 연결된 수요지 리스트

    for demand_id in connected_demands:
        # 해당 수요지의 초등학령인구 값 추출
        demand_value = data.iloc[demand_id - 1]['초등학령인구']  # demand_id는 1부터 시작하므로 1을 빼줌

        # submission 데이터 프레임에서 해당 행을 찾아 할당 학령인구 값을 업데이트
        submission.loc[(submission['수요지 id'] == f"C{demand_id}") & (submission['학교 id'] == f"F{facility_id}"), '할당 학령인구'] = demand_value

# 업데이트된 submission 데이터 프레임 출력
print(submission)


      수요지 id 학교 id  할당 학령인구
0         C1    F1        0
1         C1    F2        0
2         C1    F3        0
3         C1    F4        0
4         C1    F5        0
...      ...   ...      ...
15545   C311   F46        0
15546   C311   F47        0
15547   C311   F48        0
15548   C311   F49        0
15549   C311   F50        0

[15550 rows x 3 columns]


In [26]:
# 데이터프레임을 csv로 저장
submission.to_csv('결과1.csv', index=False)

##2.3km 이내-infeasible 상태